In [ ]:
%load_ext micropython_magic
%reload_ext micropython_magic

In [ ]:
# %mpy -s {"/dev/cu.usbmodem_fs3_1"}
%mpy -s {"/dev/cu.usbmodem1234561"}

In [ ]:
import os
from contextlib import contextmanager
from pathlib import Path

@contextmanager
def cd(new_dir: str):
    original_dir = os.getcwd()

    try:
        os.chdir(new_dir)
        yield
    finally:
        os.chdir(original_dir)

# Create mp_libs dir in case it doesn't already exist
_ = !mpremote mkdir :/mp_libs || true

# Change directories to mp_libs root dir
notebooks_dir = Path(os.getcwd())
mp_libs_dir = notebooks_dir.parent

files = []
dirs = []
with cd(mp_libs_dir):
    # Get list of mp_libs files
    for path in Path(".").rglob("*"):
        if path.is_file() and path.suffix == ".py":
            files.append(path)
        if path.is_dir():
            dirs.append(path)

    # Create all sub-dirs
    for d in dirs:
        _ = !mpremote mkdir :mp_libs/"{d}" || true

    # Copy all mp_libs files over
    for f in files:
        base_dir = f"mp_libs/{f.parent if f.parent != Path('.') else ''}/"
        !mpremote cp "{f}" :"{base_dir}"

